In [14]:
from langchain_core.runnables import RunnableSequence,RunnablePassthrough,RunnableParallel,RunnableLambda
from langchain_huggingface import HuggingFaceEmbeddings,HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

In [2]:
def getYoutubeTranscript(video_id,lang='en'):
    ytb_transcript_api = YouTubeTranscriptApi()
    transcript_list = ytb_transcript_api.fetch(video_id,[lang])
    transcripts = ' '.join([chunk.text for chunk in transcript_list])
    return transcripts

In [3]:
def format_docs(retrieved_text):
    return '\n\n'.join([doc.page_content for doc in retrieved_text])


In [4]:
video_id = 'fGo-dv3p3hY'
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [5]:
transcript = getYoutubeTranscript(video_id)
docs=splitter.create_documents([transcript])
print(docs)


[Document(metadata={}, page_content="so Hello friends welcome back to Pap coders we are back with a new video in this video we are going to learn a very New Concept which is coin like this coin is used for dependency injection we will come to know that how we are going to use dependency injection using coin and how is this coin different from dagger hild I hope so you guys have watched our playlist and this we have explained that what is this dagger H let me just redirect you to the video which I'm referring to actually you can visit papaya codas then move to the playlist section you can see a Android tutorial English playlist over here and here we have a video where you can understand dagger Hil and its implementation we have two crash courses one of them is on XML and another one is on Jetpack compos you can definitely go to through it and we also have a dedicated movie app in which I have explained what is rx Java and how we are going to make two simultaneous API calls using single 

In [6]:
embedding = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding
)
retriever = vector_store.as_retriever(
    kwargs={'k':4},
    search_type = 'similarity'
)

In [7]:
query = 'What things are dicussed in this video ?'
retrieved_context = retriever.invoke(query)
print(retrieved_context)

[Document(metadata={}, page_content="very basic UI screen okay and then we have our main activity and we have our applications setup so this is how we structure a coin method and how we are going to implement that inside our app I would suggest you guys to watch all our playlist let me just show you I would suggest you to watch this Yep this one jetpack compos movie app in this we have used coin in a very vast extent and this is literally a very good application you can refer to okay so I hope so you like this video make sure to subscribe the channel and hit the Bell icon once again and like all of our videos and do share these videos with your friends this really motivate us to make such content for you guys like we are super happy to bring you such quality content on YouTube and like this is the latest technology which you guys are learning like our channel is providing us the latest Technologies okay so I hope you like this video that's it for this video and once again subscribe and

In [8]:
prompt = PromptTemplate(
    template='''
You are a helpful assistant,
Answer ONLY from the provided transcript context.
If context is insufficient, just say you don't know

{context}
Question : {question}
''',
input_variables=['context','question']
)

In [9]:
parallel_chain = RunnableParallel(
    {
        'context': retriever | RunnableLambda(format_docs),
        'question': RunnablePassthrough()
    }
)

In [10]:
parser = StrOutputParser()

In [ ]:
parallel_chain.invoke('Who is Subham')

{'context': "get it over here okay great now what we need to do is we need to create an implementation sorry not implementation the user cases okay so let's create a another package let's name it as use case and what we will have inside this package is pretty simple we will have a class which will show the further implementation let's have it so I'm keeping this thing in mind that you have a basic idea about MV M so I'm not explaining like what is data layer what is data source what is model I hope so you have watched our videos and these things are pretty clear to you if you have any doubts please feel free to comment and you can obviously watch all our previous videos your concept would be Crystal Clear if you watch our mvvm videos okay so inside this get users use case we will have an implementation of our repository because this files requires our repository Yep this one let's import our repository and let's remove this so what we are going to do we are going to have a suspend func

In [15]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b", #google/gemma-2-2b-it
    task="conversational"
)

model = ChatHuggingFace(llm=llm)

In [16]:
main_chain = parallel_chain | prompt | model | parser

In [18]:
print(main_chain.invoke('Can you summarize the video ?'))

The video explains how to use the **Coin** library for dependency injection in an Android project.  
- It compares Coin to Dagger/Hilt and points viewers to related playlist videos.  
- It shows how to set up the main activity and application, referencing a Jetpack Compose movie‑app example.  
- The presenter then walks through adding a **GetUsersUseCase** to a ViewModel that uses a **MutableStateFlow** to expose a `List<User>`.  
- The flow is given an initial empty list, and the developer is advised to use a sealed class to represent loading, success, and error states.  
- Finally, the video touches on MVVM structure, creating a repository‑based use case, and preparing for further implementation.


In [19]:
print(main_chain.invoke(
    'Tell me what is told here and make summary of this video and where it is used and why ?'
))

**What the speaker explains**

| Topic | What’s said in the video | Why it matters / where it’s used |
|------|--------------------------|----------------------------------|
| **Koin** | “Koin is a programmatically‑designed dependency‑injection framework specifically for Kotlin… written in Kotlin only, no code generation, no reflection, uses a DSL for configuration.” | Koin is used in Android projects to provide DI without the overhead of libraries like Dagger/Hilt. By being compile‑time safe and reflection‑free it keeps the app lightweight. |
| **Key concepts** | • **Module** – a Kotlin `val` describing where objects come from.<br>• **`single`** – creates one *singleton* instance that is reused everywhere.<br>• **`factory`** – creates a new instance each time it is requested.<br>• **`viewModel`** – a special factory that gives a `ViewModel` instance that survives configuration changes.<br>• **`get()`** – a helper that automatically resolves a dependency defined in a module. | These co